## CARGA Y PROCESADO TABLAS CON DATOS FALTANTES

### Librerías necesarias

In [1]:
import numpy as np
import pandas as pd
import os

**Constante para comprobación**

In [4]:
PAISES_ESTUDIO = {'Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland','France', \
                  'Germany (until 1990 former territory of the FRG)','Greece','Hungary','Iceland','Ireland','Italy', \
                  'Latvia','Lithuania','Luxembourg','Malta','Netherlands','Norway','Poland','Portugal','Romania', \
                  'Slovakia','Slovenia','Spain','Sweden','Turkey','United Kingdom'}

### Variable Physical Activity (actividad fisica)

Cargamos archivo:

In [2]:
df_physact = pd.read_csv('./data/origen/hlth_ehis_pe9e_1_Data.csv', na_values = [': u',':'])
df_physact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
PHYSACT               300 non-null object
GEO                   300 non-null object
UNIT                  300 non-null object
TIME                  300 non-null int64
ISCED11               300 non-null object
SEX                   300 non-null object
AGE                   300 non-null object
Value                 298 non-null float64
Flag and Footnotes    0 non-null float64
dtypes: float64(2), int64(1), object(6)
memory usage: 21.2+ KB


In [10]:
df_physact.head()

,PHYSACT,GEO,UNIT,TIME,ISCED11,SEX,AGE,Value,Flag and Footnotes
0,Aerobic and muscle-strengthening,Belgium,Percentage,2014,All ISCED 2011 levels,Males,From 15 to 24 years,25.2,NaN
1,Aerobic and muscle-strengthening,Belgium,Percentage,2014,All ISCED 2011 levels,Males,From 25 to 34 years,11.5,NaN
2,Aerobic and muscle-strengthening,Belgium,Percentage,2014,All ISCED 2011 levels,Males,From 35 to 44 years,7.7,NaN
3,Aerobic and muscle-strengthening,Belgium,Percentage,2014,All ISCED 2011 levels,Males,From 45 to 64 years,5.5,NaN
4,Aerobic and muscle-strengthening,Belgium,Percentage,2014,All ISCED 2011 levels,Males,65 years or over,NaN,NaN


**Los dos faltantes aparecen para Belgica, ambos sexos, mayor de 65 años.**

In [8]:
df_physact[df_physact.Value.isnull()]

,PHYSACT,GEO,UNIT,TIME,ISCED11,SEX,AGE,Value,Flag and Footnotes
4,Aerobic and muscle-strengthening,Belgium,Percentage,2014,All ISCED 2011 levels,Males,65 years or over,NaN,NaN
9,Aerobic and muscle-strengthening,Belgium,Percentage,2014,All ISCED 2011 levels,Females,65 years or over,NaN,NaN


**Aparte no tiene datos de Holanda**

In [9]:
PAISES_ESTUDIO - set(df_physact.GEO.unique())

{'Netherlands'}

### Datos faltantes:

Busqueda de datos por internet de una fuente de confianza, arrojo datos de la oficina europea de la agencia WHO:

https://www.euro.who.int/__data/assets/pdf_file/0011/382574/netherlands-eng.pdf

https://www.euro.who.int/__data/assets/pdf_file/0010/382339/belgium-eng.pdf


**Estos datos no estan estratificados en los mismos grupos de edad de la tabla de Eurostat**


**Tomamos uno de los archivos ya procesados correctamente como referencia para las dimensiones y el orden de nuestro formato:**

In [72]:
df_referencia = pd.read_csv('./data/procesado/base_BMI.csv')
df_referencia.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 6 columns):
GEO                310 non-null object
SEX                310 non-null object
AGE                310 non-null object
BMI_Underweight    310 non-null float64
BMI_Normal         310 non-null float64
BMI_Overweight     310 non-null float64
dtypes: float64(3), object(3)
memory usage: 14.7+ KB


In [73]:
df_referencia = df_referencia[['GEO', 'SEX', 'AGE']]
df_referencia.columns

Index(['GEO', 'SEX', 'AGE'], dtype='object')

**Creamos variable auxiliar para formar la columna que añadiremos a las dimensiones, con los valores correctos y nuestro orden:**

In [74]:
# inicialización variable auxiliar, columna a añadir
col_phys_act = []

# rellenamos la variable con los datos existentes y los huecos faltantes
for registro in df_referencia.itertuples():
    
    consulta = df_physact.loc[(df_physact.GEO==registro.GEO) & (df_physact.SEX==registro.SEX) & \
                                 (df_physact.AGE==registro.AGE),'Value'].values
    
    
    if len(consulta) == 0:
        
        # dato faltante, rellenamos cno NAN para posterior procesado (corrección)
        valor = np.NaN
        
    else:
        
        # dato existente válido
        valor = consulta[0]
        
    col_phys_act.append(valor)
    

# creada la nueva columna, la añadimos:    
df_referencia['Physical_activity'] = col_phys_act

df_referencia.info()
    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 4 columns):
GEO                  310 non-null object
SEX                  310 non-null object
AGE                  310 non-null object
Physical_activity    298 non-null float64
dtypes: float64(1), object(3)
memory usage: 9.8+ KB


**Detectamos los nulos:**:

In [75]:
df_referencia[df_referencia.Physical_activity.isnull()]

,GEO,SEX,AGE,Physical_activity
4,Belgium,Males,65 years or over,NaN
9,Belgium,Females,65 years or over,NaN
180,Netherlands,Males,From 15 to 24 years,NaN
181,Netherlands,Males,From 25 to 34 years,NaN
182,Netherlands,Males,From 35 to 44 years,NaN
183,Netherlands,Males,From 45 to 64 years,NaN
184,Netherlands,Males,65 years or over,NaN
185,Netherlands,Females,From 15 to 24 years,NaN
186,Netherlands,Females,From 25 to 34 years,NaN
187,Netherlands,Females,From 35 to 44 years,NaN


- En el caso de **Belgica** para la fuente citada, tiene los datos de la franja de edad faltante pero no diferenciada por sexos. Tomando como referencia un pais del entorno con similar situación socioeconomica y extension como Dinamarca:

In [76]:
df_referencia.loc[(df_referencia.GEO=='Denmark') & (df_referencia.AGE=='65 years or over')]

,GEO,SEX,AGE,Physical_activity
34,Denmark,Males,65 years or over,20.1
39,Denmark,Females,65 years or over,21.4


Vemos que no hay diferencia considerable (solo algo más del punto porcentual) entre sexos. Por lo cual decidimos poner el mismo valor para ambos.

In [77]:
# indices de las observaciones a corregir
idx_Bel_M = df_referencia[(df_referencia.GEO=='Belgium') & (df_referencia.AGE=='65 years or over') & \
                          (df_referencia.SEX=='Males')].index.values[0]

idx_Bel_F = df_referencia[(df_referencia.GEO=='Belgium') & (df_referencia.AGE=='65 years or over') & \
                          (df_referencia.SEX=='Females')].index.values[0]


# corrección
df_referencia.loc[idx_Bel_F,'Physical_activity'] = 42
df_referencia.loc[idx_Bel_M,'Physical_activity'] = 42


In [78]:
df_referencia.loc[(df_referencia.GEO=='Belgium') & (df_referencia.AGE=='65 years or over')]

,GEO,SEX,AGE,Physical_activity
4,Belgium,Males,65 years or over,42.0
9,Belgium,Females,65 years or over,42.0


- En el caso de **Holanda** faltan todos los datos, por sexo y por grupos de edad. Los datos de la fuente citada los proporciona distinguiendo entre sexos pero con grupos de edad distintos. El grupo de edad (18-64 años) de la fuente se corresponde con 4 grupos de nuestro estudio (el quinto si es correcto). Tomando de nuevo como referencia un pais del entorno con condiciones similares como Dinamarca, vemos el reparto por grupos:

In [79]:
df_referencia.loc[(df_referencia.GEO == 'Denmark') & (df_referencia.AGE != '65 years or over') & \
                 (df_referencia.SEX == 'Males')]

,GEO,SEX,AGE,Physical_activity
30,Denmark,Males,From 15 to 24 years,48.1
31,Denmark,Males,From 25 to 34 years,28.4
32,Denmark,Males,From 35 to 44 years,22.6
33,Denmark,Males,From 45 to 64 years,21.6


Recuperamos los valores de la referencia por sexo para cada grupo de edad (todos menos el de 65+):

In [80]:
valores_Denmk_Males = df_referencia.loc[(df_referencia.GEO == 'Denmark') & (df_referencia.AGE != '65 years or over') & \
                 (df_referencia.SEX == 'Males'),'Physical_activity'].values

valores_Denmk_Females = df_referencia.loc[(df_referencia.GEO == 'Denmark') & (df_referencia.AGE != '65 years or over') & \
                 (df_referencia.SEX == 'Females'),'Physical_activity'].values



Calculamos las proporciones sobre su total que corresponden a cada uno de los cuatro grupos de edad:

In [81]:
proporc_Denmk_Males = np.round(valores_Denmk_Males / np.sum(valores_Denmk_Males),3)
proporc_Denmk_Females = np.round(valores_Denmk_Females / np.sum(valores_Denmk_Females),3)

proporc_Denmk_Males,proporc_Denmk_Females

(array([0.399, 0.235, 0.187, 0.179]), array([0.34 , 0.224, 0.213, 0.223]))

En los datos de las fuentes citadas para ambos sexos, en el único grupo de edad que engloba los del estudio, el valor es 50. Aplicamos las proporciones calculadas con la referencia sobre este valor:

In [82]:

valores_Hol_Males = 50*proporc_Denmk_Males
valores_Hol_Females = 50*proporc_Denmk_Females

valores_Hol_Males,valores_Hol_Females


(array([19.95, 11.75,  9.35,  8.95]), array([17.  , 11.2 , 10.65, 11.15]))

Localizamos los índices de las observaciones a corregir:

In [83]:
idx_Hol_Males = df_referencia.loc[(df_referencia.GEO == 'Netherlands') & (df_referencia.AGE != '65 years or over') & \
                                  (df_referencia.SEX == 'Males')].index

idx_Hol_Females = df_referencia.loc[(df_referencia.GEO == 'Netherlands') & (df_referencia.AGE != '65 years or over') & \
                                  (df_referencia.SEX == 'Females')].index

idx_Hol_Males,idx_Hol_Females

(Int64Index([180, 181, 182, 183], dtype='int64'),
 Int64Index([185, 186, 187, 188], dtype='int64'))

**Finalmente asignamos los valores:**

In [84]:
for n,idx in enumerate(list(idx_Hol_Males)):
    df_referencia.loc[idx,'Physical_activity'] = valores_Hol_Males[n]
    
    
for n,idx in enumerate(list(idx_Hol_Females)):
    df_referencia.loc[idx,'Physical_activity'] = valores_Hol_Females[n]
    
    
df_referencia.loc[df_referencia.GEO=='Netherlands']

,GEO,SEX,AGE,Physical_activity
180,Netherlands,Males,From 15 to 24 years,19.95
181,Netherlands,Males,From 25 to 34 years,11.75
182,Netherlands,Males,From 35 to 44 years,9.35
183,Netherlands,Males,From 45 to 64 years,8.95
184,Netherlands,Males,65 years or over,NaN
185,Netherlands,Females,From 15 to 24 years,17.00
186,Netherlands,Females,From 25 to 34 years,11.20
187,Netherlands,Females,From 35 to 44 years,10.65
188,Netherlands,Females,From 45 to 64 years,11.15
189,Netherlands,Females,65 years or over,NaN


**Los datos para mayores de 65 años si estan disponibles directamente en las fuentes:**

In [85]:
df_referencia.loc[184,'Physical_activity'] = 43

df_referencia.loc[189,'Physical_activity'] = 31

df_referencia.loc[df_referencia.GEO=='Netherlands']

,GEO,SEX,AGE,Physical_activity
180,Netherlands,Males,From 15 to 24 years,19.95
181,Netherlands,Males,From 25 to 34 years,11.75
182,Netherlands,Males,From 35 to 44 years,9.35
183,Netherlands,Males,From 45 to 64 years,8.95
184,Netherlands,Males,65 years or over,43.00
185,Netherlands,Females,From 15 to 24 years,17.00
186,Netherlands,Females,From 25 to 34 years,11.20
187,Netherlands,Females,From 35 to 44 years,10.65
188,Netherlands,Females,From 45 to 64 years,11.15
189,Netherlands,Females,65 years or over,31.00


**Comprobamos y guardamos resultado final:**

In [88]:
df_referencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 4 columns):
GEO                  310 non-null object
SEX                  310 non-null object
AGE                  310 non-null object
Physical_activity    310 non-null float64
dtypes: float64(1), object(3)
memory usage: 9.8+ KB


In [89]:
df_referencia.head()

,GEO,SEX,AGE,Physical_activity
0,Belgium,Males,From 15 to 24 years,25.2
1,Belgium,Males,From 25 to 34 years,11.5
2,Belgium,Males,From 35 to 44 years,7.7
3,Belgium,Males,From 45 to 64 years,5.5
4,Belgium,Males,65 years or over,42.0


In [90]:
df_referencia.to_csv('./data/procesado/base_Phys_activ.csv', index=False)

## Variable “Frequency of alcohol consumption”

En la tabla de Eurostat faltan los datos de Francia y Holanda. Buscando por internet, de nuevo fuentes de la WHO:

https://www.who.int/substance_abuse/publications/en/netherlands.pdf?ua=1

https://www.who.int/substance_abuse/publications/global_alcohol_report/profiles/fra.pdf?ua=1


Encontramos datos de porcentajes de bebedores habituales (Heavy drinkers) para ambos paises por sexo, sin distinción en grupos de edades. Haremos un procesado similar al anterior.

Como en la tabla necesitamos sumar los valores de 2 categorias de la variable independiente, y teniendo la tabla formato EHIS, lo pasamos por la función de automatización. Cargamos el resultado:

In [92]:
df_hvalc = pd.read_csv('./data/procesado/base_Freq_hv_alcohol.csv')
df_hvalc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 5 columns):
GEO                                     290 non-null object
SEX                                     290 non-null object
AGE                                     290 non-null object
Freq_hv_alcohol_At least once a week    290 non-null float64
Freq_hv_alcohol_Every month             290 non-null float64
dtypes: float64(2), object(3)
memory usage: 11.5+ KB


Para que haya concordancia en significado con los datos de las fuentes para sustituir los faltantes, debemos quedarnos con la suma de las ultimas dos columnas:

In [93]:
df_hvalc['Heavy_Drink'] = df_hvalc['Freq_hv_alcohol_At least once a week'] + df_hvalc['Freq_hv_alcohol_Every month']
df_hvalc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 6 columns):
GEO                                     290 non-null object
SEX                                     290 non-null object
AGE                                     290 non-null object
Freq_hv_alcohol_At least once a week    290 non-null float64
Freq_hv_alcohol_Every month             290 non-null float64
Heavy_Drink                             290 non-null float64
dtypes: float64(3), object(3)
memory usage: 13.7+ KB


Eliminamos las columnas que ya sumamos:

In [94]:
df_hvalc.drop(columns=['Freq_hv_alcohol_At least once a week','Freq_hv_alcohol_Every month'], inplace=True)
df_hvalc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 4 columns):
GEO            290 non-null object
SEX            290 non-null object
AGE            290 non-null object
Heavy_Drink    290 non-null float64
dtypes: float64(1), object(3)
memory usage: 9.2+ KB


**Comprobamos que no hay datos para dos países (faltan 20 observaciones para 310, con 10 por país):**

In [95]:
PAISES_ESTUDIO - set(df_hvalc.GEO.unique())

{'France', 'Netherlands'}

Preparamos para incluir los registros correspondientes a Francia y Holanda, tomando de nuevo la misma referencia para las dimensiones:

In [96]:
df_referencia = pd.read_csv('./data/procesado/base_BMI.csv')
df_referencia = df_referencia[['GEO', 'SEX', 'AGE']]

df_referencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 3 columns):
GEO    310 non-null object
SEX    310 non-null object
AGE    310 non-null object
dtypes: object(3)
memory usage: 7.4+ KB


De nuevo, para conservar nuestro formato y orden de las dimensiones, crearemos una variable auxiliar que formará la columna que añadiremos al final:

In [97]:
# inicialización variable lista auxiliar  
col_hv_drink = []

# rellenamos la variable con los datos existentes y los huecos faltantes
for registro in df_referencia.itertuples():
    
    consulta = df_hvalc.loc[(df_hvalc.GEO==registro.GEO) & (df_hvalc.SEX==registro.SEX) & \
                            (df_hvalc.AGE==registro.AGE),'Heavy_Drink'].values
    
    if len(consulta) == 0:
        
        # faltante
        valor = np.NaN
        
    else:
        
        # existente
        valor = consulta[0]
        
    col_hv_drink.append(valor)
    
# añadimos nueva columna con valores correctos a las columnas de dimensiones
df_referencia['Frequent_drink_pct'] = col_hv_drink

df_referencia.info()
    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 4 columns):
GEO                   310 non-null object
SEX                   310 non-null object
AGE                   310 non-null object
Frequent_drink_pct    290 non-null float64
dtypes: float64(1), object(3)
memory usage: 9.8+ KB


Localizamos los valores nulos:

In [98]:
df_referencia[df_referencia.Frequent_drink_pct.isnull()]

,GEO,SEX,AGE,Frequent_drink_pct
90,France,Males,From 15 to 24 years,NaN
91,France,Males,From 25 to 34 years,NaN
92,France,Males,From 35 to 44 years,NaN
93,France,Males,From 45 to 64 years,NaN
94,France,Males,65 years or over,NaN
95,France,Females,From 15 to 24 years,NaN
96,France,Females,From 25 to 34 years,NaN
97,France,Females,From 35 to 44 years,NaN
98,France,Females,From 45 to 64 years,NaN
99,France,Females,65 years or over,NaN


### Holanda: rellenar datos faltantes con los de las fuentes

Los datos diferencian entre genero (Male: 23.3%; Female: 5.6%), pero no entre grupos de edad. Al igual que como lo hicimos en el procesado de la variable anterior, tomamos como referencia los valores de un país con extensión y condiciones socioeconomicas similares, Dinamarca:


In [99]:
df_referencia.loc[(df_referencia.GEO == 'Denmark') & (df_referencia.SEX == 'Males')]

,GEO,SEX,AGE,Frequent_drink_pct
30,Denmark,Males,From 15 to 24 years,58.7
31,Denmark,Males,From 25 to 34 years,47.2
32,Denmark,Males,From 35 to 44 years,44.7
33,Denmark,Males,From 45 to 64 years,47.2
34,Denmark,Males,65 years or over,38.3


Recuperamos los valores por sexo:

In [100]:
valores_Denmk_Males = df_referencia.loc[(df_referencia.GEO == 'Denmark') & \
                                        (df_referencia.SEX == 'Males'),'Frequent_drink_pct'].values

valores_Denmk_Females = df_referencia.loc[(df_referencia.GEO == 'Denmark') & \
                                          (df_referencia.SEX == 'Females'),'Frequent_drink_pct'].values

valores_Denmk_Males, valores_Denmk_Females

(array([58.7, 47.2, 44.7, 47.2, 38.3]), array([55.7, 31.9, 20.2, 25.4, 16.8]))

Calculamos las proporciones sobre su total que corresponden a cada uno de los cinco grupos de edad:

In [101]:
proporc_Denmk_Males = np.round(valores_Denmk_Males / np.sum(valores_Denmk_Males),3)
proporc_Denmk_Females = np.round(valores_Denmk_Females / np.sum(valores_Denmk_Females),3)

proporc_Denmk_Males,proporc_Denmk_Females

(array([0.249, 0.2  , 0.189, 0.2  , 0.162]),
 array([0.371, 0.213, 0.135, 0.169, 0.112]))

En los datos de las fuentes citadas (Holanda) para ambos sexos, sin diferenciar por grupos de edad, los valores son 23.3% y 5.6% para hombres y mujeres respectivamente. Aplicamos las proporciones por grupos de edad a estos valores:

In [102]:

valores_Hol_Males = 23.3*proporc_Denmk_Males
valores_Hol_Females = 5.6*proporc_Denmk_Females

valores_Hol_Males,valores_Hol_Females


(array([5.8017, 4.66  , 4.4037, 4.66  , 3.7746]),
 array([2.0776, 1.1928, 0.756 , 0.9464, 0.6272]))

Localizamos los índices de las observaciones a completar:

In [103]:
idx_Hol_Males = df_referencia.loc[(df_referencia.GEO == 'Netherlands') & (df_referencia.SEX == 'Males')].index

idx_Hol_Females = df_referencia.loc[(df_referencia.GEO == 'Netherlands') & (df_referencia.SEX == 'Females')].index

idx_Hol_Males,idx_Hol_Females

(Int64Index([180, 181, 182, 183, 184], dtype='int64'),
 Int64Index([185, 186, 187, 188, 189], dtype='int64'))

**Finalmente asignamos los valores y comprobamos:**

In [106]:
for n,idx in enumerate(list(idx_Hol_Males)):
    df_referencia.loc[idx,'Frequent_drink_pct'] = valores_Hol_Males[n]
    
    
for n,idx in enumerate(list(idx_Hol_Females)):
    df_referencia.loc[idx,'Frequent_drink_pct'] = valores_Hol_Females[n]
    
    
df_referencia.loc[df_referencia.GEO=='Netherlands']

,GEO,SEX,AGE,Frequent_drink_pct
180,Netherlands,Males,From 15 to 24 years,5.8017
181,Netherlands,Males,From 25 to 34 years,4.6600
182,Netherlands,Males,From 35 to 44 years,4.4037
183,Netherlands,Males,From 45 to 64 years,4.6600
184,Netherlands,Males,65 years or over,3.7746
185,Netherlands,Females,From 15 to 24 years,2.0776
186,Netherlands,Females,From 25 to 34 years,1.1928
187,Netherlands,Females,From 35 to 44 years,0.7560
188,Netherlands,Females,From 45 to 64 years,0.9464
189,Netherlands,Females,65 years or over,0.6272


In [109]:
df_referencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 4 columns):
GEO                   310 non-null object
SEX                   310 non-null object
AGE                   310 non-null object
Frequent_drink_pct    300 non-null float64
dtypes: float64(1), object(3)
memory usage: 9.8+ KB


### Francia: rellenar datos faltantes con los de las fuentes

Los datos diferencian entre genero (Male: 48.7%; Female: 15.1%), pero no entre grupos de edad. De forma similar **tomaremos ahora como referencia los valores de un país con extensión y condiciones socioeconomicas similares, en este caso Alemania:**


In [110]:
df_referencia.loc[(df_referencia.GEO == 'Germany (until 1990 former territory of the FRG)')]

,GEO,SEX,AGE,Frequent_drink_pct
40,Germany (until 1990 former territory of the FRG),Males,From 15 to 24 years,50.5
41,Germany (until 1990 former territory of the FRG),Males,From 25 to 34 years,49.0
42,Germany (until 1990 former territory of the FRG),Males,From 35 to 44 years,42.1
43,Germany (until 1990 former territory of the FRG),Males,From 45 to 64 years,40.5
44,Germany (until 1990 former territory of the FRG),Males,65 years or over,34.6
45,Germany (until 1990 former territory of the FRG),Females,From 15 to 24 years,38.0
46,Germany (until 1990 former territory of the FRG),Females,From 25 to 34 years,24.6
47,Germany (until 1990 former territory of the FRG),Females,From 35 to 44 years,20.6
48,Germany (until 1990 former territory of the FRG),Females,From 45 to 64 years,23.1
49,Germany (until 1990 former territory of the FRG),Females,65 years or over,21.4


Recuperamos los valores por sexo:

In [111]:
valores_Germ_Males = df_referencia.loc[(df_referencia.GEO == 'Germany (until 1990 former territory of the FRG)') & \
                                       (df_referencia.SEX == 'Males'),'Frequent_drink_pct'].values

valores_Germ_Females = df_referencia.loc[(df_referencia.GEO == 'Germany (until 1990 former territory of the FRG)') & \
                                         (df_referencia.SEX == 'Females'),'Frequent_drink_pct'].values

valores_Germ_Males, valores_Germ_Females

(array([50.5, 49. , 42.1, 40.5, 34.6]), array([38. , 24.6, 20.6, 23.1, 21.4]))

Calculamos las proporciones sobre su total que corresponden a cada uno de los cinco grupos de edad:

In [112]:
proporc_Germ_Males = np.round(valores_Germ_Males / np.sum(valores_Germ_Males),3)
proporc_Germ_Females = np.round(valores_Germ_Females / np.sum(valores_Germ_Females),3)

proporc_Germ_Males,proporc_Germ_Females

(array([0.233, 0.226, 0.194, 0.187, 0.16 ]),
 array([0.298, 0.193, 0.161, 0.181, 0.168]))

En los datos de las fuentes citadas (Francia) para ambos sexos, sin diferenciar por grupos de edad, los valores son 48.7% y 15.1% para hombres y mujeres respectivamente. Aplicamos las proporciones de referencia a estos valores:

In [113]:

valores_Franc_Males = 48.7*proporc_Germ_Males
valores_Franc_Females = 15.1*proporc_Germ_Females

valores_Franc_Males,valores_Franc_Females


(array([11.3471, 11.0062,  9.4478,  9.1069,  7.792 ]),
 array([4.4998, 2.9143, 2.4311, 2.7331, 2.5368]))

Localizamos los índices de las observaciones a completar:

In [114]:
idx_Franc_Males = df_referencia.loc[(df_referencia.GEO == 'France') & (df_referencia.SEX == 'Males')].index

idx_Franc_Females = df_referencia.loc[(df_referencia.GEO == 'France') & (df_referencia.SEX == 'Females')].index

idx_Franc_Males,idx_Franc_Females

(Int64Index([90, 91, 92, 93, 94], dtype='int64'),
 Int64Index([95, 96, 97, 98, 99], dtype='int64'))

**Finalmente asignamos los valores y comprobamos:**

In [115]:
for n,idx in enumerate(list(idx_Franc_Males)):
    df_referencia.loc[idx,'Frequent_drink_pct'] = valores_Franc_Males[n]
    
    
for n,idx in enumerate(list(idx_Franc_Females)):
    df_referencia.loc[idx,'Frequent_drink_pct'] = valores_Franc_Females[n]
    
    
df_referencia.loc[df_referencia.GEO=='France']

,GEO,SEX,AGE,Frequent_drink_pct
90,France,Males,From 15 to 24 years,11.3471
91,France,Males,From 25 to 34 years,11.0062
92,France,Males,From 35 to 44 years,9.4478
93,France,Males,From 45 to 64 years,9.1069
94,France,Males,65 years or over,7.7920
95,France,Females,From 15 to 24 years,4.4998
96,France,Females,From 25 to 34 years,2.9143
97,France,Females,From 35 to 44 years,2.4311
98,France,Females,From 45 to 64 years,2.7331
99,France,Females,65 years or over,2.5368


In [116]:
df_referencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 4 columns):
GEO                   310 non-null object
SEX                   310 non-null object
AGE                   310 non-null object
Frequent_drink_pct    310 non-null float64
dtypes: float64(1), object(3)
memory usage: 9.8+ KB


**Con la situación de la falta de datos para los dos países resuelta, guardamos el resultado:**

In [117]:
df_referencia.to_csv('./data/procesado/base_Freq_hv_alcohol_COMPLETO.csv', index=False)